In [ ]:
Creating a class that samples from a pdf of match outcomes.

In [5]:
import collections
import itertools
import random

import tqdm

import axelrod as axl
from approximate_moran import *

In [2]:
def sample_match_outcomes(players, turns, repetitions):
    """
    Play all matches between pairs of players and return a dictionary mapping player
    names to a Pdf instance
    """
    matchups = itertools.chain(itertools.combinations(range(len(players)), 2),
                               ((i, i) for i, _ in enumerate(players)))
    match_outcomes = {}
    
    for index_pair in tqdm.tqdm(matchups):
        outcomes = []
        
        for _ in range(repetitions):
            match = axl.Match((players[index_pair[0]], players[index_pair[1]]), turns=turns)
            match.play()
            outcomes.append(match.final_score_per_turn())
            
        counts = collections.Counter(outcomes)
        pdf = Pdf(counts)
        
        player_names = tuple([str(players[i]) for i in index_pair])
        match_outcomes[player_names] = pdf
        
    return match_outcomes

Creating the match outcomes:

In [ ]:
# players = random.sample([s() for s in axl.long_run_time_strategies], 2)
players = [axl.GTFT(), axl.Random(0.6)]

turns = 200
repetitions = 100
axl.seed(0)
match_outcomes = sample_match_outcomes(players, turns=turns, repetitions=repetitions)
match_outcomes

Compare the Moran process to the approximated moran process.

In [ ]:
def update_fixation_count(moran_process, dictionary):
    """
    Play a moran process and update the 
    dictionary counting the winners
    """
    moran_process.play()
    try:
        dictionary[moran_process.winning_strategy_name] += 1
    except KeyError:
        dictionary[moran_process.winning_strategy_name] = 1

        
approx_fixation_counts = {}
exact_fixation_counts = {}
reps = 1000

for seed in range(reps):
    axl.seed(seed)
    amp = ApproximateMoranProcess(players, match_outcomes)
    update_fixation_count(amp, approx_fixation_counts)
    
    axl.seed(seed)
    mp = axl.MoranProcess(players, turns=turns)
    update_fixation_count(mp, exact_fixation_counts)

        
approx_fixation_probabilites = {k: v / reps for k, v in approx_fixation_counts.items()}
exact_fixation_probabilites = {k: v / reps for k, v in exact_fixation_counts.items()}

We see that we get the same fixation probabilities:

In [32]:
for key, value in approx_fixation_probabilites.items():
    print("{}: approximate={} exact={}".format(key, value, exact_fixation_probabilites[key]))

Random: 0.6: approximate=0.564 exact=0.571
GTFT: 0.33: approximate=0.436 exact=0.429


In [13]:
players = random.sample(axl.strategies, 10)

In [ ]:
%%time
axl.seed(100)
amp = ApproximateMoranProcess(players, match_outcomes)
amp.play()

In [15]:
%%time
axl.seed(100)
mp = axl.MoranProcess(players, turns=turns)
mp.play()

CPU times: user 472 ms, sys: 0 ns, total: 472 ms
Wall time: 470 ms


## Todo

Compute match data.

For the players in a Moran process, extract the subset of interaction pairs, compute the cumulative sums, and provide an object wrapping the sampling procedure

Compute Moran matches

In [38]:
def generate_matchups(players):
    # Want the triangular product
    for i in range(len(players)):
        for j in range(i, len(players)):
            if i == j:
                yield players[i].clone(), players[j]
            else:
                yield players[i], players[j]

def sample_match_outcomes(players, turns, repetitions, noise=0):
    """
    Play all matches between pairs of players and return a dictionary mapping player
    names to outcomes.
    """
    match_outcomes = dict()
    matchups = list(generate_matchups(players))

    for pairs in tqdm.tqdm(matchups):
        
        match = axl.Match(pairs, turns=turns, noise=noise)
        rs = repetitions
        if not match._stochastic:
            rs = 1
        outcomes = []
        for _ in range(rs):
            match.play()
            outcomes.append(match.final_score_per_turn())
            
        counts = collections.Counter(outcomes)
        
        player_names = tuple(map(str, pairs))
        match_outcomes[player_names] = counts

    return match_outcomes

In [16]:
# class Sampler(object):
#     """ToDo: Make into factory that returns subobject for specific pairs."""
    
#     def __init__(self, outcomes):
#         self.csums = {}
#         for pair, counter in outcomes.items():
#             self.csums[pair] = np.cumsum(normalize([v for k,v in sorted(counter.items())]))
#             self.outcomes[pair] = [k for k, v in sorted(counter.items())]

#     @classmethod
#     def fps(csums):
#         r = random.random()
#         for j, x in enumerate(csums):
#             if x >= r:
#                 return j

#     # Memoize?
#     def get_data(self, pair):
#         try:
#             csums = self.csums[pair]
#             outcomes = self.outcomes[pairs]
#             reverse = False
#         except KeyError:
#             try:
#                 pair_ = (pair[1], pair[0])
#                 csums = self.csums[pair]
#                 outcomes = self.outcomes[pairs]
#                 reverse = True
#             except KeyError:
#                 print("No cached data for pair {}".format(pair))
#                 raise ValueError
#         return csums, outcomes, reverse
    
#     def sample(self, pair):
#         # Need to reverse
#         csums, outcomes, reverse =  self.get_data(pair)
#         scores = self.outcomes[self.fps(csums)]
#         if reverse:
#             scores = [scores[1], scores[0]]
#         return scores

In [23]:
class Sampler(object):
    """Extracts the data for the player pair from the cached outcomes and provides
    a sampleable PDF of the score distribution."""
    
    def __init__(self, outcomes, pair_names):
        counter, reverse = self.extract_data(outcomes, pair_names)
        self.csums = np.cumsum(normalize([v for k,v in sorted(counter.items())]))
        self.scores = [k for k, v in sorted(counter.items())]
        self.reverse = reverse
        
    def extract_data(self, outcomes, pair):
        try:
            outcomes = self.outcomes[pairs]
            reverse = False
        except KeyError:
            try:
                pair_ = (pair[1], pair[0])
                outcomes = self.outcomes[pair]
                reverse = True
            except KeyError:
                print("No cached data for pair {}".format(pair))
                raise ValueError
        return outcomes, reverse
        
    @classmethod
    def fps(csums):
        r = random.random()
        for j, x in enumerate(csums):
            if x >= r:
                return j
    
    def sample(self, pair):
        # Need to reverse
        scores = self.scores[self.fps(self.csums)]
        if reverse:
            scores = [scores[1], scores[0]]
        return scores

In [ ]:
# players = [s() for s in axl.all_strategies if not
#                             s().classifier['long_run_time']]
players = [s() for s in axl.all_strategies if
           not s().classifier['stochastic']
           and not s().classifier['long_run_time']]

print(len(list(map(str, players))))

counters = sample_match_outcomes(players, turns=200, repetitions=10)

In [ ]:
counters

In [35]:
import csv
from collections import defaultdict, Counter

def write_csv(outcomes, filename="outcomes.csv"):
    writer = csv.writer(open(filename, 'w'))
    for pair, counter in outcomes.items():
        for scores, count in counter.items():
            row = [pair[0], pair[1], scores[0], scores[1], count]
            writer.writerow(row)

def read_csv(filename="outcomes.csv"):
    reader = csv.reader(open(filename))
    outcomes = defaultdict(Counter)
    for row in reader:
        p1, p2, s1, s2, count = row
        outcomes[(p1, p2)][(float(s1), float(s2))] = int(count)
    return outcomes

In [36]:
write_csv(counters)

In [ ]:
counters2 = read_csv(counters)